# Constants definitions

In [2]:
L = 10; % length = 10 m
b = 0.1; % width = 10 cm
d = 0.05; % height = 5 cm
E = 2*10^11; % Young's modulus for steel = 200 GPa = 2x10^11 Pa
I = b*d^3/12; % second moment of inertia
rho = 7850; % mass density of steel = 7850 kg/m^3
g = 9.81; % acceleration due to gravity = 9.81 m/s^2
w = rho*b*d*g; % weight of the beam per unit length (will be our f)

# Class Project Trials

In [11]:
n = 10;

for j=1:11
    printf("n: %d\t\t\t\t", n);
    h = L/n; % discretization spacing
    N = n + 1; % number of unknowns to be solved for
    A = sparse(N,N); % generating a sparse matrix

    % Define the RHS of the system
    f = -h^4/(E*I) * w * ones(N, 1);
    f(1) = 0;
    f(N) = f(N)/2;

    % Creating diagonals of the matrix
    for i=3:N - 2
        A(i,i) = 6;
        A(i,i-1) = A(i,i+1) = -4;
        A(i,i-2) = A(i,i+2) = 1; 
    endfor

    % Left end
    A(1,1) = 1;
    A(2,2) = 7;
    A(1,2) = 0;
    A(1,3) = 0;
    A(2,1) = 0;
    A(3,1) = 0;
    A(2,3) = -4;
    A(2,4) = 1;
    % Right end
    A(N,N) = 1;
    A(N-1,N-1) =  5;
    A(N-1,N) = -2;
    A(N-2,N) = 1;
    A(N, N-1) = -2;
    A(N, N-2) = 1;
    A(N-1,N-2) = -4;
    A(N-1,N-3) = 1;

    % Solve for y
    y = A\f;
    x = (0:h:L)';
    y_exact = -b*d*rho*g/(24*E*I)*x.^2.*(6.*L^2 - 4.*L*x + x.^2);
    ErrMax = max(abs(y-y_exact))
    n = n * 2;
endfor

n: 10				ErrMax =   2.3103e-002
n: 20				ErrMax =   5.7756e-003
n: 40				ErrMax =   1.4439e-003
n: 80				ErrMax =   3.6098e-004
n: 160				ErrMax =   9.0245e-005
n: 320				ErrMax =   2.2605e-005
n: 640				ErrMax =   5.7606e-006
n: 1280				ErrMax =   1.8213e-006
n: 2560				ErrMax =   2.0304e-004
n: 5120				ErrMax =   6.4788e-004
n: 10240				ErrMax =   2.1904e-002


# Class Project Trials (Penta)

In [4]:
n = 10;

for j=1:11
    printf("n: %d\t\t\t\t", n);
    h = L/n; % discretization spacing
    N = n + 1; % number of unknowns to be solved for

    % Define the RHS of the system
    f = -h^4/(E*I) * w * ones(N, 1);
    f(1) = 0;
    f(N) = f(N)/2;

    % Define the matrix of the system. Notation: d0 is the main diagonal;
    % dpn is "main diagonal + n" (superdiagonal n); dmn is "main diagonal - n" (subdiagonal n)
    d0 = 6*ones(1, N);
    dp1 = dm1 = -4*ones(1, N-1);
    dp2 = dm2 = ones(1, N-2);
    % Fixed left end
    d0(1) = 1.0; d0(2) = 7.0;
    dp1(1) = 0.0;
    dp2(1) = 0.0;
    dm1(1) = 0.0;
    dm2(1) = 0.0;
    % Free right end
    d0(N) = 1.0; d0(N-1) = 5.0;
    dp1(N-1) = -2.0;
    dp2(N-2) = 1.0;
    dm1(N-1) = -2.0;
    dm2(N-2) = 1.0;
    % Solve for y
    y = GaussElimPenta(dm2, dm1, d0, dp1, dp2, f);
    % Plots

    x = (0:h:L)';
    y_exact = -b*d*rho*g/(24*E*I)*x.^2.*(6.*L^2 - 4.*L*x + x.^2);
    ErrMax = max(abs(y-y_exact))
    n = n * 2;
endfor

n: 10				ErrMax =   2.310255000038230e-002
n: 20				ErrMax =   5.775637502192676e-003
n: 40				ErrMax =   1.443909349510442e-003
n: 80				ErrMax =   3.609771821433405e-004
n: 160				ErrMax =   9.025203271706417e-005
n: 320				ErrMax =   2.258974779412171e-005
n: 640				ErrMax =   5.456730395092535e-006
n: 1280				ErrMax =   2.688451615906473e-005
n: 2560				ErrMax =   5.210340448424944e-006
n: 5120				ErrMax =   1.661748340157310e-003
n: 10240				ErrMax =   4.423001108697200e-003


# Class Project Trials (Non-Ficitious nodes)

In [3]:
n = 10;

for j=1:11
    printf("n: %d\t\t\t\t", n);
    h = L/n; % discretization spacing
    N = n + 1; % number of unknowns to be solved for

    % Define the RHS of the system
    f = -h^4/(E*I) * w * ones(N, 1);
    f(1) = f(2) = f(N-1) = f(N) = 0;

    format long
    A = sparse(N,N);

    A(1,1) = 1;
    A(2,1) = -3;
    A(2,2) = 4;
    A(2,3) = -1;


    % Creating diagonals of the matrix
    for i=3:N - 2
        A(i,i) = 6;
        A(i,i-1) = A(i,i+1) = -4;
        A(i,i-2) = A(i,i+2) = 1; 
    endfor

    A(N-1,N) = 2;
    A(N-1,N-1) = -5;
    A(N-1,N-2) = 4;
    A(N-1,N-3) = -1;

    A(N,N) = 5;
    A(N,N-1) = -18;
    A(N,N-2) = 24;
    A(N,N-3) = -14;
    A(N,N-4) = 3;

    % Solve for y
    y = A\f;

    x = (0:h:L)';
    y_exact = -b*d*rho*g/(24*E*I)*x.^2.*(6.*L^2 - 4.*L*x + x.^2);
    ErrMax = max(abs(y-y_exact))
    n = n * 2;
endfor

n: 10				ErrMax =   2.772305999983748e-002
n: 20				ErrMax =   6.353201249225116e-003
n: 40				ErrMax =   1.516104928419804e-003
n: 80				ErrMax =   3.700023166035571e-004
n: 160				ErrMax =   9.137374992018721e-005
n: 320				ErrMax =   2.264982619282208e-005
n: 640				ErrMax =   5.368584908627128e-006
n: 1280				ErrMax =   3.240280728178391e-006
n: 2560				ErrMax =   1.393908128073384e-004
n: 5120				ErrMax =   1.594388222612153e-004
n: 10240				ErrMax =   7.004777839347831e-004


# Sauer First Edition Trials

In [12]:
n = 10;
for j=1:11
    printf("n: %d\t\t\t\t", n);
    h = L/n; % discretization spacing
    N = n; % number of unknowns to be solved for
    A = sparse(N,N); % generating a sparse matrix

    % Define the RHS of the system
    f = -h^4/(E*I) * w * ones(N, 1);

    % Creating diagonals of the matrix
    for i=3:N - 2
        A(i,i) = 6;
        A(i,i-1) = A(i,i+1) = -4;
        A(i,i-2) = A(i,i+2) = 1; 
    endfor

    % Leftside
    A(1,1) = 12;
    A(1,2) = -6;
    A(1,3) = (4/3);
    A(2,1) = A(2,3) = -4;
    A(2,2) = 6;    
    A(2,4) = 1;

    % right endpoint
    A(N,N) = (12/25);
    A(N-1,N) = (-43/25);
    A(N,N-1) = (-24/25);
    A(N-1,N-1) = (111/25);
    A(N,N-2) = (12/25);
    A(N-1,N-2) = (-93/25);
    A(N-1,N-3) = 1;
    % Solve for y
    y = A\f;
    x = (h:h:L)';
    y_exact = -b*d*rho*g/(24*E*I)*x.^2.*(6.*L^2 - 4.*L*x + x.^2);
    ErrMax = max(abs(y-y_exact))
    n = n * 2;
endfor

n: 10				ErrMax =   6.6227e-001
n: 20				ErrMax =   3.1959e-001
n: 40				ErrMax =   1.5690e-001
n: 80				ErrMax =   7.7730e-002
n: 160				ErrMax =   3.8685e-002
n: 320				ErrMax =   1.9297e-002
n: 640				ErrMax =   9.6371e-003
n: 1280				ErrMax =   4.8098e-003
n: 2560				ErrMax =   2.4800e-003
n: 5120				ErrMax =   4.1564e-004
n: 10240				ErrMax =   1.3762e-002


# Sauer First Edition Trials - Penta

In [13]:
n = 10;
format short e
for j=1:11
    printf("n: %d\t\t\t\t", n);
    h = L/(n+1); % discretization spacing
    N = n+1; % number of unknowns to be solved for 

    format short e
    % Define the RHS of the system
    f = -h^4/(E*I) * w * ones(N, 1);

    % Define the matrix of the system. Notation: d0 is the main diagonal;
    % dpn is "main diagonal + n" (superdiagonal n); dmn is "main diagonal - n" (subdiagonal n)
    d0 = 6*ones(1, N);
    dp1 = dm1 = -4*ones(1, N-1);
    dp2 = dm2 = ones(1, N-2);
    % Fixed left end
    d0(1) = 12;
    dp1(1) = -6;
    dp2(1) = (4/3);
    dm1(1) = -4;
    dm2(1) = 1;
    % Free right end
    d0(N) = (12/25); d0(N-1) = (111/25);
    dp1(N-1) = (-43/25);
    dp2(N-2) = 1.0;
    dm1(N-1) = (-24/25);
    dm1(N-2) = (-93/25);
    dm2(N-2) = (12/25);
    % Solve for y
    y = GaussElimPenta(dm2, dm1, d0, dp1, dp2, f);

    % Plots
    x = (h:h:L)';
    y_exact = -b*d*rho*g/(24*E*I)*x.^2.*(6.*L^2 - 4.*L*x + x.^2);
    ErrMax = max(abs(y-y_exact))
    n = n * 2;
endfor

n: 10				ErrMax =   5.9825e-001
n: 20				ErrMax =   3.0384e-001
n: 40				ErrMax =   1.5301e-001
n: 80				ErrMax =   7.6762e-002
n: 160				ErrMax =   3.8443e-002
n: 320				ErrMax =   1.9237e-002
n: 640				ErrMax =   9.6216e-003
n: 1280				ErrMax =   4.8111e-003
n: 2560				ErrMax =   2.4270e-003
n: 5120				ErrMax =   1.3870e-003
n: 10240				ErrMax =   1.0745e-002


# Sauer Second Edition Trials

In [14]:
n = 10;
format short e
for j=1:11
    printf("n: %d\t\t\t\t", n);
    h = L/n; % discretization spacing
    N = n; % number of unknowns to be solved for
    A = sparse(N,N);
    % Define the RHS of the system
    f = -h^4/(E*I) * w * ones(N, 1);
    % Creating diagonals of the matrix
    for i=3:N - 2
        A(i,i) = 6;
        A(i,i-1) = A(i,i+1) = -4;
        A(i,i-2) = A(i,i+2) = 1; 
    endfor

    % Leftside
    A(1,1) = 16;
    A(1,2) = -9;
    A(1,3) = (8/3);
    A(1,4) = (-1/4);
    A(2,1) = A(2,3) = -4;
    A(2,2) = 6;
    A(2,4) = 1;

    % Rightside
    A(N,N-3) = -(12/17);
    A(N,N-2) = (96/17);
    A(N,N-1) = -(156/17);
    A(N,N) = (72/17);
    A(N-1,N) = -(28/17);
    A(N-1,N-1) = (72/17);
    A(N-1,N-2) = -(60/17);
    A(N-1,N-3) = (16/17);
    % Finding y
    y = A\f;

    % Plots
    x = (h:h:L)';
    y_exact = -b*d*rho*g/(24*E*I)*x.^2.*(6.*L^2 - 4.*L*x + x.^2);
    ErrMax = max(abs(y-y_exact))
    n = n * 2;
endfor

n: 10				ErrMax =   1.5854e-013
n: 20				ErrMax =   9.6945e-013
n: 40				ErrMax =   4.7085e-011
n: 80				ErrMax =   3.2013e-010
n: 160				ErrMax =   3.7130e-009
n: 320				ErrMax =   8.9597e-008
n: 640				ErrMax =   1.9621e-007
n: 1280				ErrMax =   1.0212e-006
n: 2560				ErrMax =   4.0676e-005
n: 5120				ErrMax =   9.0815e-004
n: 10240				ErrMax =   9.4137e-003
